In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
device

device(type='cuda')

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
train_datasets = torchvision.datasets.MNIST(root = '../../data/',
                                            train = True,
                                            transform = transforms.ToTensor(),
                                            download = True)
test_datasets = torchvision.datasets.MNIST(root = '../../data/',
                                           train = False,
                                           transform = transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset = train_datasets,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_datasets,
                                           batch_size = batch_size,
                                           shuffle = False)

In [17]:
class ConvNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = ConvNet(num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [21]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 ==0:
            print('Epoch[{}/{}], Step[{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch[1/5], Step[100/600], Loss: 0.0044
Epoch[1/5], Step[200/600], Loss: 0.0151
Epoch[1/5], Step[300/600], Loss: 0.0290
Epoch[1/5], Step[400/600], Loss: 0.0397
Epoch[1/5], Step[500/600], Loss: 0.0166
Epoch[1/5], Step[600/600], Loss: 0.0202
Epoch[2/5], Step[100/600], Loss: 0.0230
Epoch[2/5], Step[200/600], Loss: 0.0037
Epoch[2/5], Step[300/600], Loss: 0.0007
Epoch[2/5], Step[400/600], Loss: 0.0054
Epoch[2/5], Step[500/600], Loss: 0.0076
Epoch[2/5], Step[600/600], Loss: 0.0242
Epoch[3/5], Step[100/600], Loss: 0.0030
Epoch[3/5], Step[200/600], Loss: 0.0163
Epoch[3/5], Step[300/600], Loss: 0.0194
Epoch[3/5], Step[400/600], Loss: 0.0125
Epoch[3/5], Step[500/600], Loss: 0.0059
Epoch[3/5], Step[600/600], Loss: 0.0898
Epoch[4/5], Step[100/600], Loss: 0.0057
Epoch[4/5], Step[200/600], Loss: 0.0074
Epoch[4/5], Step[300/600], Loss: 0.0011
Epoch[4/5], Step[400/600], Loss: 0.0013
Epoch[4/5], Step[500/600], Loss: 0.0055
Epoch[4/5], Step[600/600], Loss: 0.0088
Epoch[5/5], Step[100/600], Loss: 0.0033


In [22]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {}%'.format(100*correct/total))

Test Accuracy of the model on the 10000 test images: 99.07%


In [23]:
torch.save(model.state_dict(), 'model.ckpt')